<u><h1 align='center'>Lab1: Building a spam filter for text messages</h1></u>

<br/>

Mobile phone spam also known as (unsolicited messages, especially advertising),
directed at the text messaging or other communications services of mobile phones
or smartphones. <br/>
Fighting SMS spam is complicated by several factors (compared
to Internet email), including the lower rate of SMS spam, which has allowed many
users and service providers to ignore the issue, and the limited availability of
mobile phone spam-filtering software.<br/>
<br/>
In this lab, we’ll be looking at a text classification problem using python where we
sort spam messages from ham


# 0. Imports

In [117]:
import pandas as pd
#numpy
import numpy as np
from sklearn.model_selection import train_test_split
import re
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.metrics import accuracy_score, recall_score, precision_score

In [48]:
#!pip install autocorrect
from autocorrect import spell


     |████████████████████████████████| 19.1MB 3.1MB/s eta 0:00:01    |███                             | 1.8MB 1.2MB/s eta 0:00:15
  Created wheel for autocorrect: filename=autocorrect-0.4.4-cp37-none-any.whl size=19134812 sha256=6369f5eb73c91da328c39ad3210d907425c07f775137f0a014f26ba50992cc85
  Stored in directory: /home/abir/.cache/pip/wheels/e5/80/6d/feed99c22f3f15b0b6881569dc9021f7e94b6464aa0688d819
Successfully built autocorrect


# 1. Prepare train and test datasets:

In [5]:
# import data

spams = pd.read_table("SMSSpamCollection.txt",sep="\t",header=0)
messages = pd.read_table("SMSSpamCollection.txt",sep="\t")
#type of the object
print("type of the object spams: ",type(spams))
#size of the dataset
print('size of the dataset: ',spams.shape)

type of the object spams:  <class 'pandas.core.frame.DataFrame'>
size of the dataset:  (5572, 2)


/home/abir/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.
/home/abir/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead.
  


In [17]:
spams.head()

,classe,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


# 2. Text pre-processing:

In [81]:
#add spell checker since we nooticed that there are some mistakes( two words with the same meaning are represented by two keywords in our dictionary)
corpus = []
for i in range(0,len(messages['message'])):
    smsDocument = spell(re.sub('[^a-zA-Z]',' ',messages['message'][i]))
    smsDocument = smsDocument.lower()
    smsDocument = smsDocument.split()
    lem = WordNetLemmatizer()
    smsDocument = [lem.lemmatize(word) for word in smsDocument if not word in set(stopwords.words('english'))]
    
    smsDocument = ' '.join(smsDocument)
    corpus.append(smsDocument)
    
smsDocuments = pd.Series(corpus)

autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is depr

autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is depr

autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is depr

autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is depr

autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is depr

...

autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is depr

autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is depr

autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is depr

autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is depr

autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is deprecated, use autocorrect.Speller instead
autocorrect.spell is depr

In [93]:
#subdivision into train and test sets
#stratify: same distribution in training and testing sets
spams['message']=smsDocuments
spamsTrain, spamsTest=train_test_split(spams[["message",'classe']],train_size=3572,random_state=1,stratify=spams['classe'])

/home/abir/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [94]:
#frequency distribution of the class attribute
#train set
freqTrain = pd.crosstab(index=spamsTrain["classe"],columns="count")
print("Frequency distribution of the class attribute For Training set:\n\n",freqTrain/freqTrain.sum())
#test set
freqTest = pd.crosstab(index=spamsTest["classe"],columns="count")
print("\nFrequency distribution of the class attribute For Test set:\n\n",freqTest/freqTest.sum())

Frequency distribution of the class attribute For Training set:

 col_0      count
classe          
ham     0.865901
spam    0.134099

Frequency distribution of the class attribute For Test set:

 col_0   count
classe       
ham     0.866
spam    0.134


# 3. Feature extraction:
   ## a) Bag of Words Model

In [95]:
#bag of words
parseur = CountVectorizer()
#create the document term matrix
XTrain = parseur.fit_transform(spamsTrain['message'])

In [96]:
#number of tokens
print(len(parseur.get_feature_names()))

5183


In [97]:
#list of tokens
print(parseur.get_feature_names())

['aaooooright', 'ab', 'aba', 'abbey', 'abdomen', 'aberdeen', 'ability', 'able', 'abnormally', 'aboutas', 'absolutely', 'abstract', 'abt', 'abuse', 'abuser', 'ac', 'acantha', 'acc', 'accent', 'accept', 'access', 'accessible', 'accident', 'accidentally', 'accommodation', 'accomodate', 'according', 'accordingly', 'account', 'accounting', 'acct', 'ache', 'acid', 'ack', 'acknowledgement', 'acl', 'across', 'act', 'acted', 'actin', 'acting', 'activate', 'active', 'activity', 'actor', 'actual', 'actually', 'ad', 'adam', 'add', 'addamsfa', 'added', 'addicted', 'adding', 'addle', 'address', 'adeem', 'adenase', 'adjustable', 'admin', 'administrator', 'admirer', 'admission', 'admit', 'adod', 'adore', 'adoring', 'adr', 'adult', 'advance', 'adventure', 'advice', 'advise', 'advising', 'advisor', 'ae', 'aee', 'aeronautics', 'aeroplane', 'aet', 'affair', 'affection', 'affectionate', 'afford', 'afghanistan', 'afraid', 'african', 'aft', 'afternoon', 'afterwards', 'ag', 'agar', 'age', 'agency', 'agent', '

In [98]:
#transform the sparse matrix into a numpy matrix
mdtTrain = XTrain.toarray()
#type of the matrix
print('type of the matrix: ',type(mdtTrain))
#size of the matrix
print('size of the matrix: ',mdtTrain.shape)

type of the matrix:  <class 'numpy.ndarray'>
size of the matrix:  (3572, 5183)


In [99]:
#frequency of the terms
freq_mots = np.sum(mdtTrain,axis=0)
print(freq_mots)

[1 3 1 ... 1 1 1]


In [100]:
#arg sort
index = np.argsort(freq_mots)
print(index)

[   0 2896 2898 ... 4837 1820  663]


In [102]:
#print the terms and their frequency
imp = {'terme':np.asarray(parseur.get_feature_names())[index],'freq':freq_mots[index]}
print(pd.DataFrame(imp))

                terme  freq
0         aaooooright     1
1               motto     1
2               mouse     1
3         movietrivia     1
4                mozo     1
5                 mph     1
6               mquiz     1
7             msgrcvd     1
8                 mtn     1
9            mudyadhu     1
10              mufti     1
11           motorial     1
12           muhammad     1
13           multiple     1
14           munsters     1
15              mural     1
16           murdered     1
17           murderer     1
18              musca     1
19               mush     1
20              mushy     1
21          musickers     1
22              musth     1
23         multimedia     1
24              motor     1
25           motivate     1
26             motion     1
27          miserable     1
28             misfit     1
29    missunderstding     1
...               ...   ...
5153              lor    98
5154             take    99
5155            later    99
5156            stil

## b) TF-IDF Model

In [107]:
tfidf = TfidfTransformer(norm="l2")
tfidf.fit(mdtTrain)
print ("IDF:",tfidf.idf_)

IDF: [8.48801368 7.7948665  8.48801368 ... 8.48801368 8.48801368 8.48801368]


In [108]:
tf_idf_matrix = tfidf.transform(mdtTrain)
print (tf_idf_matrix.todense())
tf_idf_matrix = tf_idf_matrix.toarray()


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# 4. Modeling & prediction & Evaluation:

In [151]:
#create the document term matrix for test set
mdtTest = parseur.transform(spamsTest['message'])
mdtTest = mdtTest.toarray()

print('size of the matrix: ',mdtTest.shape)

size of the matrix:  (2000, 5183)


In [149]:
classifiers = [
    KNeighborsClassifier(),
    LogisticRegression(solver='liblinear'),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=10),
    AdaBoostClassifier(),
    GaussianNB()]

In [152]:
results_list = []


for clf in classifiers:
    clf_name = clf.__class__.__name__
    print("="*30)
    print(clf_name)
    clf.fit(mdtTrain,spamsTrain['classe'])

        
    predTest = clf.predict(mdtTest)
    acc = metrics.accuracy_score(spamsTest['classe'],predTest)
    r=metrics.recall_score(spamsTest['classe'],predTest,predTest,pos_label='spam')
    p=metrics.precision_score(spamsTest['classe'],predTest,predTest,pos_label='spam')
    results_list.append((clf_name, acc*100,r*100,p*100))


results_df = pd.DataFrame(results_list,columns=["Classifier", "Accuracy", "Recall",'Precision'])
results_df.set_index('Classifier',inplace=True)

KNeighborsClassifier
LogisticRegression
DecisionTreeClassifier
RandomForestClassifier
AdaBoostClassifier
GaussianNB


In [154]:
results_df.head(6)

,Accuracy,Recall,Precision
Classifier,,,
KNeighborsClassifier,91.50,36.567164,100.000000
LogisticRegression,97.80,84.701493,98.695652
DecisionTreeClassifier,96.50,81.716418,91.250000
RandomForestClassifier,96.75,77.611940,97.652582
AdaBoostClassifier,96.65,81.343284,92.765957
GaussianNB,87.10,86.567164,51.101322


In [159]:
#create the document term matrix for test set

mdtTest = tfidf.transform(mdtTest)
mdtTest = mdtTest.toarray()

print('size of the matrix: ',mdtTest.shape)

size of the matrix:  (2000, 5183)


In [161]:
results_list = []


for clf in classifiers:
    clf_name = clf.__class__.__name__
    print("="*30)
    print(clf_name)
    clf.fit(tf_idf_matrix,spamsTrain['classe'])

        
    predTest = clf.predict(mdtTest)
    acc = metrics.accuracy_score(spamsTest['classe'],predTest)
    r=metrics.recall_score(spamsTest['classe'],predTest,predTest,pos_label='spam')
    p=metrics.precision_score(spamsTest['classe'],predTest,predTest,pos_label='spam')
    results_list.append((clf_name, acc*100,r*100,p*100))


results_df = pd.DataFrame(results_list,columns=["Classifier", "Accuracy", "Recall",'Precision'])
results_df.set_index('Classifier',inplace=True)

KNeighborsClassifier
LogisticRegression
DecisionTreeClassifier
RandomForestClassifier
AdaBoostClassifier
GaussianNB


In [164]:
results_df.head(6)

,Accuracy,Recall,Precision
Classifier,,,
KNeighborsClassifier,90.15,26.492537,100.000000
LogisticRegression,90.70,31.716418,96.590909
DecisionTreeClassifier,91.30,51.492537,75.824176
RandomForestClassifier,94.50,60.074627,98.170732
AdaBoostClassifier,93.55,53.358209,97.278912
GaussianNB,85.80,80.597015,48.214286
